In [ ]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate

from dosipy.utils.integrate import elementwise_dblquad

In [ ]:
def v(t, s):
    """Parametric surface in 3-D:
    v(t, s) = (t + 1) i + (s) j + (s^2 - t^2 + 1) k
    """
    return np.c_[t + 1,
                 s,
                 s ** 2 - t ** 2 + 1]

def v_t(t, s):
    """First derivative of parametric surface, v, wrt t:
    v_t(t, s) = i  + (-2 t) k
    """
    return np.c_[np.ones_like(t),
                 np.zeros_like(t),
                 - 2 * t]

def v_s(t, s):
    """First derivative of parametric surface, v, wrt s:
    v_s(t, s) = j + (2 s) k
    """
    return np.c_[np.zeros_like(s),
                 np.ones_like(s),
                 2 * s]

def F(x, y, z):
    """Vector field that passes through V(t, s):
    """
    return np.c_[np.ones_like(x),
                 np.ones_like(y),
                 np.ones_like(z)]

In [ ]:
# integration domain
t_a, t_b = -2, 2
s_a, s_b = -2, 2

# integration points
t = np.linspace(t_a, t_b, 101)
s = np.linspace(s_a, s_b, 101)
T, S = np.meshgrid(t, s)
V = v(T.ravel(), S.ravel())

In [ ]:
# surface v
fig = plt.figure(figsize=(6, 6))
ax = plt.axes(projection ='3d')
ax.scatter(*V.T, s=0.5)
ax.set(xlabel='$t$', ylabel='$s$', zlabel='$v(t, s)$')
ax.view_init(25, 125)

In [ ]:
# vector field F across the surface v
fig = plt.figure(figsize=(6, 6))
ax = plt.axes(projection ='3d')
scaler = 50
rnd_idx = V.shape[0] * np.random.random_sample(scaler)
rnd_idx = np.asarray(rnd_idx, dtype=np.int32)
ax.scatter(*V.T, s=0.5)
ax.quiver(*V[rnd_idx].T, *F(*V[rnd_idx].T).T, color='red', length=1, label='$\\vecF(v(t, s))$')
ax.set(xlabel='$t$', ylabel='$s$', zlabel='$v(t, s)$')
ax.legend()
ax.view_init(25, 125)

In [ ]:
# computing unit normals
V_T = v_t(T.ravel(), S.ravel())
V_S = v_s(T.ravel(), S.ravel())
n = np.cross(V_T, V_S)

In [ ]:
# vector field F across the surface v
fig = plt.figure(figsize=(6, 6))
ax = plt.axes(projection ='3d')
ax.scatter(*V.T, s=0.5)
ax.quiver(*V[rnd_idx].T, *F(*V[rnd_idx].T).T, color='r', length=1,
          label='$\\vecF(v(t, s))$')
ax.quiver(*V[rnd_idx].T, *n[rnd_idx].T, color='k', normalize=True, length=1,
          label='$\\vecn$')
ax.set(xlabel='$t$', ylabel='$s$', zlabel='$v(t, s)$')
ax.legend()
ax.view_init(25, 125)

In [ ]:
points = np.c_[T.ravel(), S.ravel()]  # 2-D set of points
values = np.sum(F(*V.T) * n, axis=1)  # integrand = F(V(t, s)) · n(x, y, z)
I_exact = 16

In [ ]:
# vector field F across the surface v
fig = plt.figure(figsize=(6, 6))
ax = plt.axes(projection ='3d')
cs = ax.scatter(*V.T, s=10, c=values, )
cbar = fig.colorbar(cs, shrink=0.55, pad=0.15)
cbar.ax.set_ylabel('$\\vecF \\cdot \\vec n$')
ax.set(xlabel='$t$', ylabel='$s$', zlabel='$v(t, s)$')
ax.view_init(25, 125)

In [ ]:
# vector field F across the surface v
fig = plt.figure(figsize=(6, 5))
ax = plt.axes()
cs = ax.scatter(*points.T, s=10, c=values)
cbar = fig.colorbar(cs)
cbar.ax.set_ylabel('$\\vecF \\cdot \\vec n$')
ax.set(xlabel='$t$', ylabel='$s$');

In [ ]:
I_approx = elementwise_dblquad(points, values, degree=21)

round(I_approx, 5)